# 인테리어 & 건축 QA 데이터 - LLM fine tuning

- 모델 아키텍쳐
    - base model: ko-llama
    - model 업데이트 수준: QLoRa - low rank & 4bit fine tuning
- data:
    - 데이터 출처: https://dacon.io/competitions/official/236216/overview/description
    - query & answer 그대로 사용
    - 추가적인 data augmentation 미시행
- 리소스: GPU P100
- 모듈 버전: torch==2.1.0 필요

In [ ]:
! nvidia-smi -L # 윈도우 GPU 사용량 cmd 명령어

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-f14d39be-e673-11bf-e0e1-84806a125892)


In [ ]:
!pip install torch==2.1.0

  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl (209.8 MB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.24 requires torch==2.2.0, but you have torch 2.1.0 which is incompatible.


In [ ]:
%%time
# !pip install -qU bitsandbytes transformers datasets accelerate loralib einops # xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.2.0 whic

In [ ]:
%%time
# !pip install -q -U git+https://github.com/huggingface/peft.git

In [ ]:
# !pip list

# Load Data

In [ ]:
import pandas as pd
import os

root_path = os.getcwd()+"/drive/MyDrive/contest/dacon_hansol_llm"

train_data = pd.read_csv(os.path.join(root_path, "train.csv"))
train_data.sample(3)

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
104,TRAIN_104,KMEW 세라믹 지붕재가 뭐야?,KMEW 세라믹 지붕재는 어떤 특징을 가지고 있나요?,마감재,KMEW 세라믹 지붕재는 산화티타늄처리를 한 지붕재입니다.,"KMEW 세라믹 지붕재는 일본의 기업인 KMEW에서 개발한 지붕재로, 산화티타늄 처...","KMEW 세라믹 지붕재는 산화티타늄 처리를 한 경질화 세라믹 지붕재로, 우수한 내구...","KMEW 세라믹 지붕재는 산화티타늄 처리를 한 지붕재로, 햇빛에 빛나는 효과와 함께...","KMEW 세라믹 지붕재는 특별한 산화티타늄 처리를 통해 제조된 지붕재로, 건축물의 ..."
13,TRAIN_013,면진구조가 뭐야?,면진구조에 대해 좀 더 자세히 알 수 있을까요?,건축구조,면진구조란 면진 장치를 사용해 지반과 건물을 분리시키는 방법을 의미합니다.,면진구조는 지반과 건물을 분리시키기 위해 면진 장치를 사용하는 구조를 말합니다. 이...,면진구조란 지반과 건물을 분리시키기 위해 면진 장치를 사용하는 방법을 말합니다. 이...,면진구조란 지반과 건물을 분리시키기 위해 면진 장치를 사용하는 구조를 말합니다. 이...,"면진구조는 건물의 지반과의 연결을 분리시키는 구조로, 지진이나 지반의 이동에 따른 ..."
490,TRAIN_490,벽에 구멍을 막는 방법은?,"벽에 구멍을 막는데 사용할 수 있는 다양한 방법이 있는데, 어떤 방법이 가장 효과적...",시공,"구멍을 메우기 위해 합성 먼지 또는 석고를 사용하고, 필요하면 패팅을 적용합니다.","벽에 구멍을 막는 가장 일반적인 방법은 석고나 합성 재료로 구멍을 메우고, 필요하다...","벽에 구멍을 메우기 위해서는 먼저 구멍 주변을 깨끗하게 솔로 닦아내고, 모래지거나 ...",벽에 구멍을 메우는 방법은 다음과 같습니다. 1. 먼저 구멍 주변을 깨끗이 닦고 분...,"벽에 구멍을 메우기 위해 먼저 합성 먼지나 석고를 사용하여 구멍을 메우고, 이후에는..."


In [ ]:
train_data['category'].value_counts()

마감재       272
인테리어      123
시공        111
마감하자       60
건축구조       31
기타         27
타 마감하자     20
Name: category, dtype: int64

# Import Modules(Base Pretrained Model & Fine-Tuning)

- 모듈 간 버전 충돌 주의: torch==2.1.0 요구됨
- 충분한 RAM 메모리 필요

## a. LoRa

In [ ]:
import os
import bitsandbytes as bnb
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset

from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training, # 4bir Qlora
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## b. QLora(4bit) & ko-LLAMA
- BitsAndBytesConfig: 양자화

In [ ]:
%%time

model_id = "beomi/llama-2-ko-7b" # 한국어 llama # "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/965M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/231 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

CPU times: user 29.2 s, sys: 36.2 s, total: 1min 5s
Wall time: 6min 20s


In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(52000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    

# Config - LoRa

In [ ]:
%%time

# target_modules: 파인튜닝 추가할 레이어 대상이 너무 많으면 너무 오래걸림..!
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"], #["query_key_value"], llama2에는 없음
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters()

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(52000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False


# 전처리

In [ ]:
from tqdm import tqdm

# 데이터 포맷팅 및 토크나이징
formatted_data = []
for _, row in tqdm(train_data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:

            input_text = row[q_col]
            target_text = row[a_col]
            inputs = tokenizer(input_text,
                                 add_special_tokens=True,
                                 max_length = 512,
                                 truncation=True,
                                  padding="max_length")
            targets = tokenizer(target_text,
                                 add_special_tokens=True,
                                 max_length = 512,
                                 truncation=True,
                                 padding="max_length")
            formatted_data.append({'input_ids': inputs['input_ids'],
                                    'attention_mask': inputs['attention_mask'],
                                    'labels': targets['input_ids'],
                                    })
print('Done.')

644it [00:10, 58.83it/s]

Done.


In [ ]:
input_text

'AD, PD에 면한 벽체 결로에 대한 대책은 어떤 것이 있나요?'

In [ ]:
target_text

'AD, PD에 면한 벽체의 결로에 대한 대책으로는 단열재를 미실하게 시공하여 결로가 생기는 벽체의 표면 온도를 노점온도 이상으로 유지해주는 것이 중요합니다. 결로가 발생하는 것을 방지하기 위해서는 적절한 수온조절과 단열재 시공이 필요합니다. 또한, 실내 습도를 적정 수준으로 유지하고 외부에서의 열이나 수분이 벽체에 스며들지 않도록 주의해야 합니다. AD, PD에 면한 벽체 결로에 대한 대책으로는 건물의 외부와 내부 온도 대조를 잘 조절하여 결로 발생을 최소화하는 것이 필요합니다.'

In [ ]:
targets

{'input_ids': [1854, 15, 13945, 279, 2483, 295, 11834, 11373, 888, 38259, 991, 7357, 5156, 15722, 13016, 643, 31968, 31280, 1566, 888, 8670, 16608, 11834, 11373, 42917, 1561, 2592, 948, 943, 33050, 13989, 3314, 12312, 1406, 20136, 17, 888, 8670, 22133, 1941, 35901, 9188, 21669, 422, 1174, 39848, 408, 15722, 562, 31280, 266, 28302, 17, 2604, 15, 8792, 8740, 2592, 13915, 11070, 20270, 21902, 7555, 1669, 1008, 422, 2408, 11834, 673, 279, 27409, 38364, 13024, 7038, 1244, 1637, 17, 861, 39, 15, 13945, 279, 2483, 295, 11834, 673, 888, 38259, 991, 7357, 5156, 5244, 299, 1414, 9991, 4211, 43008, 390, 13031, 800, 16625, 1566, 888, 294, 2213, 284, 46073, 1406, 28302, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
len(formatted_data)

6440

In [ ]:
len(formatted_data[0]['input_ids'])

512

# Train(Fine-Tuning)

In [ ]:
OUTPUT_DIR = root_path

In [ ]:
training_args = transformers.TrainingArguments(
    auto_find_batch_size=True,
    num_train_epochs=4,
    learning_rate=2e-4,
#     bf16=True, # ValueError: Your setup doesn't support bf16/gpu. -> fp16=True로 바꿈
    fp16=True,
    save_total_limit=1,
    logging_steps=20,
    output_dir=OUTPUT_DIR,
    overwrite_output_dir =True,
    # evaluation_strategy="steps", # 이 기능 쓰려면 valid set 넣어야 함
    save_strategy='epoch'
)

  # args = TrainingArguments(
  #     output_dir=f"./out_fold{i}",
  #     overwrite_output_dir = 'True',
  #     evaluation_strategy="steps",
  #     eval_steps=40,
  #     logging_steps = 40,
  #     learning_rate = 5e-5,
  #     per_device_train_batch_size=8,
  #     per_device_eval_batch_size=8,
  #     num_train_epochs=10,
  #     seed=0,
  #     save_total_limit = 1,
  #     # report_to = "none",
  # #     logging_steps = 'epoch',
  #     load_best_model_at_end=True,
  #     save_strategy = "no"
  # )

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=formatted_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

Step,Training Loss
20,4.679400
40,4.526200
60,3.839700
80,3.581900
100,3.072600
120,3.056200
140,2.743100
160,2.617600
180,2.566100
200,2.109800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
trainer.save_model(root_path+"/fine_tuned_model")